# API Keys

In [7]:
from config import HUGGING_FACE_API_KEY, PINECONE_API_KEY

# Text Collector

In [8]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

loader = TextLoader('../Behlol.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)

In [9]:
documents

[Document(metadata={'source': '../Behlol.txt'}, page_content='حکایت شکار رفتن بهلول و هارون\nروزی خلیفه هارون الرشید و جمعی از درباریان به شکار رفته بودند. بهلول با آنها بود در شکارگاه آهویی نمودار شد.\nخلیفه تیری به سوی آهو انداخت ولی به هدف نخورد. بهلول گفت احسنت !!!\nخلیفه غضبناک شد و گفت مرا مسخره می کنی ؟\nبهلول جواب داد :احسنت من برای آهو بود که خوب فرار نمود.\n\nحکایت زیبای بهلول و سوداگر\nروزی سوداگری بغدادی از بهلول سوال نمود من چه بخرم تا منافع زیاد ببرم؟ بهلول جواب داد آهن و پنبه.\nآن مرد رفت و مقداری آهن و پنبه خرید و انبار نمود اتفاقا" پس از چند ماهی فروخت و سود فراوان برد.\nباز روزی به بهلول بر خورد. این دفعه گفت بهلول دیوانه من چه بخرم تا منافع ببرم؟ بهلول این دفعه گفت پیاز بخر و هندوانه.\nسوداگر این دفعه رفت و سرمایه خود را تمام پیاز خرید و هندوانه انبار نمود و پس از مدت کمی تمام پیاز و هندوانه های او پوسید و از بین رفت و ضرر فراوان نمود.\nفوری به سراغ بهلول رفت و به او گفت در اول که از تو مشورت نموده، گفتی آهن بخر و پنبه، نفعی برده. ولی دفعه دوم این چه پیشنهادی بود کرد

In [10]:
docs

[Document(metadata={'source': '../Behlol.txt'}, page_content='حکایت شکار رفتن بهلول و هارون\nروزی خلیفه هارون الرشید و جمعی از درباریان به شکار رفته بودند. بهلول با آنها بود در شکارگاه آهویی نمودار شد.\nخلیفه تیری به سوی آهو انداخت ولی به هدف نخورد. بهلول گفت احسنت !!!\nخلیفه غضبناک شد و گفت مرا مسخره می کنی ؟\nبهلول جواب داد :احسنت من برای آهو بود که خوب فرار نمود.'),
 Document(metadata={'source': '../Behlol.txt'}, page_content='حکایت زیبای بهلول و سوداگر\nروزی سوداگری بغدادی از بهلول سوال نمود من چه بخرم تا منافع زیاد ببرم؟ بهلول جواب داد آهن و پنبه.\nآن مرد رفت و مقداری آهن و پنبه خرید و انبار نمود اتفاقا" پس از چند ماهی فروخت و سود فراوان برد.\nباز روزی به بهلول بر خورد. این دفعه گفت بهلول دیوانه من چه بخرم تا منافع ببرم؟ بهلول این دفعه گفت پیاز بخر و هندوانه.\nسوداگر این دفعه رفت و سرمایه خود را تمام پیاز خرید و هندوانه انبار نمود و پس از مدت کمی تمام پیاز و هندوانه های او پوسید و از بین رفت و ضرر فراوان نمود.\nفوری به سراغ بهلول رفت و به او گفت در اول که از تو مشورت نموده، گفتی آ

# Embedings

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/tmp/ipykernel_13462/3077754740.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/home/amin/ICT/rag-with-crawler/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

# Vector Store

In [ ]:
PINECONE_INDEX_NAME = 'parrot'

In [ ]:
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Initialize Pinecone client
pinecone_client = PineconeClient(
    api_key= PINECONE_API_KEY,
)

index_name = PINECONE_INDEX_NAME

# print(pinecone_client.list_indexes().names())

if index_name not in pinecone_client.list_indexes().names():

  # Initialize Pinecone Client
  pinecone_client.create_index(
      name=PINECONE_INDEX_NAME,
      metric='cosine',
      dimension=768,
      spec=ServerlessSpec(
                  cloud='aws',
                  region='us-east-1'
              )
  )

  # Set UP the link and Upload data
  vector_store = PineconeVectorStore.from_documents(
        docs,
        index_name=index_name,
        embedding=embeddings
    )

else:

  # Set UP the link
  vector_store = PineconeVectorStore.from_existing_index(
      index_name=index_name,
      embedding=embeddings
    )

# LLM

In [ ]:
from langchain.llms import HuggingFaceHub

# Define the repo ID and connect to Mixtral model on Huggingface
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(
  repo_id=repo_id,
  model_kwargs={"temperature": 0.8, "top_k": 50},
  huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
)

<ipython-input-11-98c975d13931>:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


'hello world!\n#include <stdio.h>\n\nint main()\n{\n    char s[100];\n    printf("Enter a string: ");\n    scanf("%s",s);\n    printf("String: %s\\n", s);\n\n    int i;\n    for (i=0; s[i]; i++){\n        printf("%c\\n", s[i]);\n    }\n\n    return 0;\n}'

In [ ]:
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
LLMResult(generations=[[Generation(text='hi,\n\ni use a webservice for my app in delphi and it works fine. my app only has one form (mainform) and the webservice is instantiated in the form as an object "ws".\n\nmy question is: if the user closes the app, how long does the instantiated object "ws" (webservice) live, and do i have to do something to ensure that "ws" is destroyed?\n\nthanks,\nnagara\n\nPS: i am a newbie and this is my first post, so if i made any mistakes, please let me know, and i will correct them.')]], llm_output=None, run=[RunInfo(run_id=UUID('2a2f0212-b120-4571-a372-8d1d0ed81154'))], type='LLMResult')

# Prompt

In [ ]:
from langchain import PromptTemplate

# template = """
# You are a fortune teller. These Human will ask you a questions about their life.
# Use following piece of context to answer the question.
# If you don't know the answer, just say you don't know.
# Keep the answer within 2 sentences and concise.

# Context: {context}
# Question: {question}
# Answer:

# """

template = """
لطفا بر اساس داستان مطرح شده به سوال پاسخ بده

داستان: {context}
سوال: {question}
جواب:

"""

prompt = PromptTemplate(
  template=template,
  input_variables=["context", "question"]
)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": vector_store.as_retriever(),  "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

In [ ]:
tmp = input()
result = rag_chain.invoke(tmp)
print(result)

In [ ]:
result = rag_chain.invoke('آیا بهلول به هارون الرشید گفت آفرین؟')
print(result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



لطفا بر اساس داستان مطرح شده به سوال پاسخ بده

داستان: [Document(id='3a7fe21a-4436-44a7-8a3c-7eb4f61e5977', metadata={'source': './Behlol.txt'}, page_content='حکایت شکار رفتن بهلول و هارون\nروزی خلیفه هارون الرشید و جمعی از درباریان به شکار رفته بودند. بهلول با آنها بود در شکارگاه آهویی نمودار شد.\nخلیفه تیری به سوی آهو انداخت ولی به هدف نخورد. بهلول گفت احسنت !!!\nخلیفه غضبناک شد و گفت مرا مسخره می کنی ؟\nبهلول جواب داد :احسنت من برای آهو بود که خوب فرار نمود.'), Document(id='5d4cd2c5-5e16-4249-92da-a7072e63f1b8', metadata={'source': './Behlol.txt'}, page_content='حکایت جالب و داستان ازدواج بهلول\nجوانی نزد بهلول آمد و پرسید:من از بدبختی دائم در فکرم که چه خاکی به سر کنم! سبب چیست که پدر می گوید:زن بگیر، درست میشود!\nبهلول گفت:حکمت آن است که پس از ازدواج دوتایی فکر خواهید کرد که چه خاکی به سر کنید.'), Document(id='54dec108-3f8b-4d5a-9780-9759c754a52d', metadata={'source': './Behlol.txt'}, page_content='حکایت زیبای بهلول و سوداگر\nروزی سوداگری بغدادی از بهلول سوال نمود من چه بخرم تا م

In [ ]:
result = rag_chain.invoke('از نگاه بهلول سود در خرید چیست؟')
print(result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



لطفا بر اساس داستان مطرح شده به سوال پاسخ بده

داستان: [Document(id='5d4cd2c5-5e16-4249-92da-a7072e63f1b8', metadata={'source': './Behlol.txt'}, page_content='حکایت جالب و داستان ازدواج بهلول\nجوانی نزد بهلول آمد و پرسید:من از بدبختی دائم در فکرم که چه خاکی به سر کنم! سبب چیست که پدر می گوید:زن بگیر، درست میشود!\nبهلول گفت:حکمت آن است که پس از ازدواج دوتایی فکر خواهید کرد که چه خاکی به سر کنید.'), Document(id='3a7fe21a-4436-44a7-8a3c-7eb4f61e5977', metadata={'source': './Behlol.txt'}, page_content='حکایت شکار رفتن بهلول و هارون\nروزی خلیفه هارون الرشید و جمعی از درباریان به شکار رفته بودند. بهلول با آنها بود در شکارگاه آهویی نمودار شد.\nخلیفه تیری به سوی آهو انداخت ولی به هدف نخورد. بهلول گفت احسنت !!!\nخلیفه غضبناک شد و گفت مرا مسخره می کنی ؟\nبهلول جواب داد :احسنت من برای آهو بود که خوب فرار نمود.'), Document(id='54dec108-3f8b-4d5a-9780-9759c754a52d', metadata={'source': './Behlol.txt'}, page_content='حکایت زیبای بهلول و سوداگر\nروزی سوداگری بغدادی از بهلول سوال نمود من چه بخرم تا م